# Filtrer les données d'OpenFoodFacts

Dans ce notebook, nous allons filtrer les données d'OpenFoodFacts pour pouvoir les exploiter.

La base de données OpenFoodFacts est ouverte à toutes les contributions, même incomplètes, et sans restriction géographique (produits vendus dans certains pays seulement).
Un filtrage sur les lignes nous permettra de ne conserver que les produits vendus en France et dont la fiche est suffisamment remplie.

Nous pouvons charger les données d'OpenFoodFacts avec les optimisations du notebook précédent.

In [1]:
import pandas as pd
# pour les graphes
import seaborn as sns
sns.set()

In [10]:
# TODO modifier le chemin vers le fichier CSV si nécessaire
CSV_FILE = '../data/en.openfoodfacts.org.products.csv'

In [14]:
# charger les types de données qu'on a spécifiés et exportés dans le notebook 1
DTYPE_FILE = '../data/dtype.txt'  # TODO chemin?
with open(DTYPE_FILE) as f:
    dtype = eval(f.read())
# pour vérification, on affiche les types de données explicites
list(dtype.items())[:5]

[('origins', 'category'),
 ('origins_tags', 'category'),
 ('first_packaging_code_geo', 'category'),
 ('cities_tags', 'category'),
 ('purchase_places', 'category')]

In [1]:
# TODO charger le fichier CSV : toutes les lignes, colonnes utilisables, types de données spécifiés

## Produits vendus en France

Quels sont le ou les champs contenant cette information?

Nous allons créer un masque sur les lignes du DataFrame :
`True` si le produit est vendu en France, `False` sinon.

La méthode [.str.contains()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html) permet de chercher une sous-chaîne dans une colonne de chaînes de caractères : 

In [3]:
# créer un masque pour filtrer les produits vendus en France


Nous pouvons maintenant connaître le nombre de produits vendus en France.

In [4]:
# utiliser value_counts() ou describe()


Nous allons nous concentrer sur ce sous-ensemble des données en ne conservant que les lignes correspondantes dans le DataFrame.

In [6]:
# appliquer le masque au DataFrame pour filtrer ses lignes


## Fiches de qualité suffisante

Toutes (ou presque toutes?) les bases de données de grande taille souffrent de problèmes de qualité : incohérences entre valeurs, valeurs manquantes, erreurs...
Les bases de données contributives, étant alimentées par de nombreux fournisseurs, sont particulièrement sujettes à ces problèmes.
Il est de bonne pratique d'écarter les entrées que nous suspectons d'être incomplètes ou de mauvaise qualité, afin d'éviter que ces entrées n'agissent comme des anomalies (*outliers*) qui compromettent l'analyse ou l'entraînent vers des conclusions erronées.

De nombreuses fiches produits dans OpenFoodFacts ne sont remplies que très partiellement.
Un premier filtre assez simple consiste à ne conserver que les fiches où le nom et la marque du produit sont renseignés.
Les fiches ne répondant pas à critère peuvent être écartées avec [dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [7]:
# écarter les lignes où le nom ou la marque manque


À chaque fiche produit d'OpenFoodFacts sont associées une ou plusieurs étiquettes d'états ('states') qui décrivent le niveau de complétude et de qualité des données de la fiche.

Quels sont ces états? À quoi correspondent-ils?

Sur quelle(s) pages ou dans quel(s) fichier(s) avez-vous trouvé ces informations?

Nous ne conserverons que les entrées marquées "complètes".
Filtrez les données en utilisant [.str.contains()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html).

In [8]:
# conserver les fiches complètes


## Export du jeu de données filtré

Nous avons maintenant un jeu de données de dimensions plus modestes, en nombre de colonnes et de lignes.
Nous pouvons l'exporter dans un fichier CSV avec [to_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas.DataFrame.to_csv), afin de pouvoir l'explorer plus facilement dans les prochains notebooks.

In [9]:
# faire un dump CSV du jeu de données filtré
